This notebook is done following 
* [Building text classifier with Differential Privacy](https://github.com/pytorch/opacus/blob/main/tutorials/building_text_classifier.ipynb)
* [Fine-tuning with custom datasets](https://huggingface.co/transformers/v3.4.0/custom_datasets.html#seq-imdb)

# Libraries
https://huggingface.co/docs/transformers/training

## Install

In [1]:
!pip install datasets
!pip install transformers
!pip install opacus

     |████████████████████████████████| 325 kB 5.5 MB/s 
     |████████████████████████████████| 212 kB 12.9 MB/s 
     |████████████████████████████████| 1.1 MB 45.2 MB/s 
     |████████████████████████████████| 136 kB 49.3 MB/s 
     |████████████████████████████████| 77 kB 4.7 MB/s 
     |████████████████████████████████| 127 kB 45.9 MB/s 
     |████████████████████████████████| 94 kB 2.2 MB/s 
     |████████████████████████████████| 271 kB 46.4 MB/s 
     |████████████████████████████████| 144 kB 18.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 4.0 MB 5.2 MB/s 


## Import

In [2]:
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from torch.optim import AdamW
from transformers import get_scheduler
from torch.utils.data import TensorDataset, DataLoader
import torch
from torch.nn.utils.rnn import pad_sequence
import gc
from opacus.utils.batch_memory_manager import BatchMemoryManager

from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

## Set drive if needed

In [ ]:
from google.colab import drive

use_drive = True

if use_drive:
  drive.mount('/content/drive')
  COLABROOTDIR="/content/drive/My Drive/Projects/DP/"
  os.environ["COLABROOTDIR"] = COLABROOTDIR

## [Check GPU footprint](https://stackoverflow.com/questions/59789059/gpu-out-of-memory-error-message-on-google-colab)

In [4]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed

def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
if len(GPUs) > 0:
  gpu = GPUs[0]
  printm()
else:
  print("No running GPU")

No running GPU


## Get device

In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cpu


# Dataset

## Download

First, we need to download the dataset. https://huggingface.co/datasets/tweets_hate_speech_detection

In [97]:
from datasets import load_dataset, load_from_disk

# if you have processed and saved the dataset
# dataset = load_from_disk(COLABROOTDIR + 'tweet-dataset')

tweets_dataset = load_dataset('tweets_hate_speech_detection')

for key in tweets_dataset.keys():
  print(key, tweets_dataset[key].shape)

04/27/2022 04:39:58:WARNING:Using custom data configuration default
04/27/2022 04:39:58:WARNING:Reusing dataset tweets_hate_speech_detection (/root/.cache/huggingface/datasets/tweets_hate_speech_detection/default/0.0.0/c6b6f41e91ac9113e1c032c5ecf7a49b4e1e9dc8699ded3c2d8425c9217568b2)


  0%|          | 0/1 [00:00<?, ?it/s]

train (31962, 2)


In [101]:
num_labels = 2
tweets_datasets = tweets_dataset['train'].train_test_split(test_size=0.1)
tweets_datasets = tweets_datasets.rename_column("tweet", "text")

tweets_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'tweet'],
        num_rows: 28765
    })
    test: Dataset({
        features: ['label', 'tweet'],
        num_rows: 3197
    })
})

In [102]:
# dataset = load_dataset("yelp_review_full")
# imdb_dataset = load_dataset("imdb")

# for key in imdb_dataset.keys():
#   print(key, imdb_dataset[key].shape)

# # positive or negative review
# num_labels = 2

# print(imdb_dataset["train"][100])

# lengths = []
# for i in ['train', 'test']:
#   for item in imdb_dataset[i]:
#     lengths.append(len(item['text']))

In [105]:
lengths = []
for i in ['train', 'test']:
  for item in tweets_datasets[i]:
    lengths.append(len(item['text']))

In [106]:
import pandas as pd
df = pd.DataFrame({'Lengths':lengths})
df.describe()

,Lengths
count,31962.000000
mean,84.560822
std,29.491059
min,10.000000
25%,62.000000
50%,88.000000
75%,107.000000
max,273.000000


## Gender swapping
Generate couple of test dataset versions switching gender based pronouns

In [107]:
import re
male2female=u"""maleS femaleS, maleness femaleness,
him her, himself herself, his her, his hers, he she,
Mr Mrs, Mister Missus, Ms Mr, Master Miss, Master Mistress,
uncleS auntS, nephewS nieceS, sonS daughterS, grandsonS granddaughterS,
brotherS sisterS, man woman, men women, boyS girlS, paternal maternal,
grandfatherS grandmotherS,
husband wife, husbands wives, fatherS motherS, bridegroomS brideS, widowerS widowS,
KingS QueenS,PrinceS PrincessES,
Lord Lady, Lords Ladies,ladS lassES, sir madam, gentleman lady, gentlemen ladies,
godS goddessES, heroS heroineS, landlord landlady, landlords landladies, 
manservantS maidservantS, actorS actressES,
boyfriendS girlfriendS, dogS bitchES, daddy mommy, dadS momS"""
 
re_newline=re.compile(r",[ \n]*")
male2female_splitted=[ token.split(" ") for token in re_newline.split(male2female) ]
 
re_plural=re.compile("E*S$")
re_ES=re.compile("ES$")
 
def gen_pluralize(m,f):
# do plurals first 
  yield re_plural.sub("",m),re_plural.sub("",f)
  yield re_ES.sub("es",m),re_ES.sub("es",f)
  yield re_plural.sub("s",m),re_plural.sub("s",f)
 
def gen_capitalize_pluralize(m,f):
  for m,f in gen_pluralize(m,f):
    yield m.capitalize(), f.capitalize()
    yield m,f

# converts male pronouns to female and female to male
def gen_switch(male_to_female=True, female_to_male=True):
  switch={}
  words=[]

  for male,female in male2female_splitted:
    for xy, xx in gen_capitalize_pluralize(male,female):
    # for xy, xx in gen_pluralize(male,female):
      if male_to_female and xy not in switch: 
        switch[xy]=xx
        words.append(xy)
      if female_to_male and xx not in switch: 
        switch[xx]=xy
        words.append(xx)

  words="|".join(words)
  re_word = re.compile(r"\b("+words+r")\b")
  return re_word, switch

def rev_gender(text, re_word, switch):
  text=re_word.split(text)
  return "".join([ word+switch[gen] for word,gen in zip(text[::2],text[1::2])]) + text[-1]

In [110]:
dataset = tweets_datasets
# dataset['test'] = dataset['test'].shuffle(seed=2022).shard(5, index=0)

In [109]:
dataset['test_gender_swapped'] = dataset['test']
dataset['test_male_to_female'] = dataset['test']
dataset['test_female_to_male'] = dataset['test']

In [112]:
re_word_both, switch_both = gen_switch(male_to_female=True, female_to_male=True)
re_word_f2m, switch_f2m = gen_switch(male_to_female=False, female_to_male=True)
re_word_m2f, switch_m2f = gen_switch(male_to_female=True, female_to_male=False)

for index in tqdm(range(len(dataset['test']['text']))):
    text = dataset['test']['text'][index]

    dataset['test_gender_swapped']['text'][index] = rev_gender(text, re_word_both, switch_both)
    dataset['test_female_to_male']['text'][index] = rev_gender(text, re_word_f2m, switch_f2m)
    dataset['test_male_to_female']['text'][index] = rev_gender(text, re_word_m2f, switch_m2f)

  0%|          | 0/3197 [00:00<?, ?it/s]

In [ ]:
# save the dataset in drive since it takes much time to do this processing
# dataset.save_to_disk(COLABROOTDIR + 'tweet-dataset')

## Tokenizer

In [62]:
from transformers import BertConfig, BertTokenizer

model_name = "bert-base-cased"
config = BertConfig.from_pretrained(
    model_name,
    num_labels=2,
)
tokenizer = BertTokenizer.from_pretrained(
    model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

## Prepare the data
Before we begin training, we need to preprocess the data and convert it to the format our model expects.

(Note: it'll take 5-10 minutes to run on a laptop)

In [113]:
MAX_SEQ_LENGTH = 256

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", max_length=MAX_SEQ_LENGTH, truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/29 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

04/27/2022 04:52:18:WARNING:Loading cached processed dataset at /root/.cache/huggingface/datasets/tweets_hate_speech_detection/default/0.0.0/c6b6f41e91ac9113e1c032c5ecf7a49b4e1e9dc8699ded3c2d8425c9217568b2/cache-449688f3665327d0.arrow
04/27/2022 04:52:18:WARNING:Loading cached processed dataset at /root/.cache/huggingface/datasets/tweets_hate_speech_detection/default/0.0.0/c6b6f41e91ac9113e1c032c5ecf7a49b4e1e9dc8699ded3c2d8425c9217568b2/cache-449688f3665327d0.arrow
04/27/2022 04:52:19:WARNING:Loading cached processed dataset at /root/.cache/huggingface/datasets/tweets_hate_speech_detection/default/0.0.0/c6b6f41e91ac9113e1c032c5ecf7a49b4e1e9dc8699ded3c2d8425c9217568b2/cache-449688f3665327d0.arrow


In [124]:
# str value gets error during data loading step
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

In [125]:
# select a smaller subset for faster debugging
small_train_dataset = tokenized_datasets["train"].shuffle(seed=2022).select(range(10))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=2022).select(range(10))


test_swapped_dataset = tokenized_datasets['test_gender_swapped'].shuffle(seed=2022).select(range(10))
test_m2f_dataset = tokenized_datasets['test_male_to_female'].shuffle(seed=2022).select(range(10))
test_f2m_dataset = tokenized_datasets['test_female_to_male'].shuffle(seed=2022).select(range(10))

04/27/2022 04:54:49:WARNING:Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/tweets_hate_speech_detection/default/0.0.0/c6b6f41e91ac9113e1c032c5ecf7a49b4e1e9dc8699ded3c2d8425c9217568b2/cache-5aae6a806f8f7a32.arrow
04/27/2022 04:54:49:WARNING:Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/tweets_hate_speech_detection/default/0.0.0/c6b6f41e91ac9113e1c032c5ecf7a49b4e1e9dc8699ded3c2d8425c9217568b2/cache-5aae6a806f8f7a32.arrow
04/27/2022 04:54:49:WARNING:Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/tweets_hate_speech_detection/default/0.0.0/c6b6f41e91ac9113e1c032c5ecf7a49b4e1e9dc8699ded3c2d8425c9217568b2/cache-5aae6a806f8f7a32.arrow


# Model

BERT (Bidirectional Encoder Representations from Transformers) is a state of the art approach to various NLP tasks. It uses a Transformer architecture and relies heavily on the concept of pre-training.

We'll use a pre-trained BERT-base model, provided in huggingface [transformers](https://github.com/huggingface/transformers) repo. It gives us a pytorch implementation for the classic BERT architecture, as well as a tokenizer and weights pre-trained on a public English corpus (Wikipedia).

Please follow these [installation instrucitons](https://github.com/huggingface/transformers#installation) before proceeding.

In [67]:
# https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertForSequenceClassification
from transformers import BertForSequenceClassification

def load_pretrained_model(model_name, config):
    model = BertForSequenceClassification.from_pretrained(model_name, config=config)

    trainable_layers = [model.bert.encoder.layer[-1], model.bert.pooler, model.classifier]
    total_params = 0
    trainable_params = 0

    for p in model.parameters():
      p.requires_grad = False
      total_params += p.numel()

    for layer in trainable_layers:
      for p in layer.parameters():
          p.requires_grad = True
          trainable_params += p.numel()
          total_params += p.numel()

    print(f"Total parameters count: {total_params}") # ~108M
    print(f"Trainable parameters count: {trainable_params}") # ~7M

    return model

In [126]:
model = load_pretrained_model(model_name, config)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Total parameters count: 115991812
Trainable parameters count: 7680002


# Data loader

In [70]:
BATCH_SIZE = 16
MAX_PHYSICAL_BATCH_SIZE = 2

In [127]:
# train_dataloader = get_dataloader(small_train_dataset, BATCH_SIZE)
# test_dataloader = get_dataloader(small_eval_dataset, BATCH_SIZE)

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(small_eval_dataset, batch_size=BATCH_SIZE)

test_swapped_dataloader = DataLoader(test_swapped_dataset, batch_size=BATCH_SIZE)
test_m2f_dataloader = DataLoader(test_m2f_dataset, batch_size=BATCH_SIZE)
test_f2m_dataloader = DataLoader(test_f2m_dataset, batch_size=BATCH_SIZE)

# Training

In [118]:
EPOCHS = 3
EPSILON = 7.5
DELTA = 1 / len(train_dataloader) # Parameter for privacy accounting. Probability of not achieving privacy guarant
NOISE_MULTIPLIER = 0.1
LEARNING_RATE = 1e-3
MAX_GRAD_NORM = 1

In [128]:
model = model.to(device)

# Set the model to train mode (HuggingFace models load in eval mode)
model = model.train()
# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, eps=1e-8)

# criterion
loss_function = torch.nn.CrossEntropyLoss()

## Evaluation cycle

In [74]:
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

# https://huggingface.co/docs/datasets/metrics
def calculate_result(labels, preds):
    return {
        'accuracy': np.round(accuracy_score(labels, preds), 4),
        'f1': np.round(f1_score(labels, preds), 4),
        'auc': np.round(roc_auc_score(labels, preds), 4)
    }

def evaluate(model, test_dataloader):    
    model.eval()

    losses, total_preds, total_labels = [], [], []
    
    for batch in test_dataloader:
        inputs = {k: v.to(device) for k, v in batch.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            
        loss = outputs[0]
        
        preds = np.argmax(outputs.logits.detach().cpu().numpy(), axis=1)
        labels = inputs['labels'].detach().cpu().numpy()
        
        losses.append(loss.item())
        total_preds.extend(preds)
        total_labels.extend(labels)
    
    model.train()
    return np.mean(losses), calculate_result(total_labels, total_preds), total_preds

## Privacy Engine

In [75]:
from opacus import PrivacyEngine

privacy_engine = PrivacyEngine()

In [129]:
# model, optimizer, train_dataloader = privacy_engine.make_private_with_epsilon(
#     module=model,
#     optimizer=optimizer,
#     data_loader=train_dataloader,
#     target_delta=DELTA,
#     target_epsilon=EPSILON, 
#     epochs=EPOCHS,
#     max_grad_norm=MAX_GRAD_NORM,
# )

model, optimizer, train_dataloader = privacy_engine.make_private(
    module=model,
    optimizer=optimizer,
    data_loader=train_dataloader,
    noise_multiplier=NOISE_MULTIPLIER,
    max_grad_norm=MAX_GRAD_NORM,
    poisson_sampling=False,
)

## Train

In [130]:
import gc
gc.collect()

3940

In [131]:
for epoch in range(1, EPOCHS+1):
    losses, total_preds, total_labels = [], [], []

    with BatchMemoryManager(
        data_loader=train_dataloader, 
        max_physical_batch_size=MAX_PHYSICAL_BATCH_SIZE, 
        optimizer=optimizer
    ) as memory_safe_data_loader:
        for step, data in enumerate(tqdm(memory_safe_data_loader)):
            optimizer.zero_grad()

            inputs = {k: v.to(device) for k, v in data.items()}
            outputs = model(**inputs) # output = loss, logits, hidden_states, attentions

            targets = data['labels'].to(device, dtype = torch.long)
            # loss = loss_function(outputs.logits, targets)
            loss = outputs[0]

            loss.backward()
            optimizer.step()

            losses.append(loss.item())

            preds = np.argmax(outputs.logits.detach().cpu().numpy(), axis=1)
            labels = targets.detach().cpu().numpy()
            total_preds.extend(preds)
            total_labels.extend(labels)
           

    train_loss = np.mean(losses)
    train_result = calculate_result(np.array(total_labels), np.array(total_preds))

    eps = privacy_engine.get_epsilon(DELTA)
    eval_loss, eval_result, eval_preds = evaluate(model, test_dataloader)

    print(
      f"Epoch: {epoch} | "
      f"ɛ: {eps:.2f} |"
      f"Train loss: {train_loss:.3f} | "
      f"Train result: {train_result} |\n"
      f"Eval loss: {eval_loss:.3f} | "
      f"Eval result: {eval_result} | "
    )

  0%|          | 0/8 [00:00<?, ?it/s]

Epoch: 1 | ɛ: 216.65 |Train loss: 0.889 | Train result: {'accuracy': 0.1, 'f1': 0.1818, 'auc': 0.5} |
Eval loss: 0.556 | Eval result: {'accuracy': 0.8, 'f1': 0.0, 'auc': 0.5} | 


  0%|          | 0/8 [00:00<?, ?it/s]

Epoch: 2 | ɛ: 271.65 |Train loss: 0.507 | Train result: {'accuracy': 0.9, 'f1': 0.0, 'auc': 0.5} |
Eval loss: 0.505 | Eval result: {'accuracy': 0.8, 'f1': 0.0, 'auc': 0.5} | 


  0%|          | 0/8 [00:00<?, ?it/s]

Epoch: 3 | ɛ: 326.65 |Train loss: 0.391 | Train result: {'accuracy': 0.9, 'f1': 0.0, 'auc': 0.5} |
Eval loss: 0.603 | Eval result: {'accuracy': 0.8, 'f1': 0.0, 'auc': 0.5} | 


In [132]:
# just check if the model is underfitting
sum(total_preds), sum(total_labels), len(total_labels)

(0, 1, 10)

In [133]:
swap_loss, swap_result, swap_preds = evaluate(model, test_swapped_dataloader)
m2f_loss, m2f_result, m2f_preds = evaluate(model, test_m2f_dataloader)
f2m_loss, f2m_result, f2m_preds = evaluate(model, test_f2m_dataloader)

In [134]:
swap_result, m2f_result, f2m_result

({'accuracy': 0.8, 'auc': 0.5, 'f1': 0.0},
 {'accuracy': 0.8, 'auc': 0.5, 'f1': 0.0},
 {'accuracy': 0.8, 'auc': 0.5, 'f1': 0.0})

In [136]:
preds_df = pd.DataFrame({'text':tokenizer.batch_decode(small_eval_dataset['input_ids'], skip_special_tokens=True), 
  'labels':small_eval_dataset['labels'], 'eval_pred':eval_preds,'swap_pred': swap_preds,
  'm2f_pred': m2f_preds,'f2m_pred': f2m_preds
})

In [137]:
preds_df

,text,labels,eval_pred,swap_pred,m2f_pred,f2m_pred
0,secrets of a # marriage and a # family,0,0,0,0,0
1,# msnbc # cnn # amjoy joe biden :'i want to th...,1,0,0,0,0
2,indoor hockey stas back up tomorrow # stressre...,0,0,0,0,0
3,why are white people # expats when the rest of...,1,0,0,0,0
4,this weeks # wednesdaywisdom from one of my al...,0,0,0,0,0
5,stop comparing and wishing ðbe happy with what...,0,0,0,0,0
6,rough as fuck. # nomakeup # nosoul # nodrugs #...,0,0,0,0,0
7,when your just happy just because ððððð # life...,0,0,0,0,0
8,my saturday morning mood. # dancing # dancelif...,0,0,0,0,0
9,shout out to my # babydaddy # fathersday i mus...,0,0,0,0,0


In [142]:
if use_drive:
  preds_df.to_csv(COLABROOTDIR + 'results.csv', index=False)